# NER Fine Tuning with Hugging Face

This notebook evaluates using Hugging Face's NER pipeline for fine tuning on the labelled articles. The Specter2 model is intended to be used as a BERT based encoder model with a token classification head.

Specter2 model card on HF hub: https://huggingface.co/allenai/specter2

In [4]:
# for training on colab
from google.colab import drive
import os, sys

drive.mount('/content/drive')

os.chdir(os.path.join("drive", "MyDrive", "Colab Notebooks", "MetaExtractor"))

labelled_file_path = os.path.join(
    os.getcwd(), 
    os.pardir, 
    os.pardir, 
    "Projects", 
    "591-fossils-in-the-literature", 
    "data", 
    "labelled", 
    "2023-05-23_labelling-export"
  )

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install transformers datasets evaluate accelerate seqeval 

In [4]:
import os, sys

import pandas as pd
import numpy as np
from datasets import load_dataset

from transformers import AutoTokenizer

# ensure src is in path
sys.path.append(os.path.join(os.getcwd(), os.pardir, os.pardir, os.pardir, os.pardir))

# use autoreload
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
os.getcwd()

'c:\\Users\\tyand\\OneDrive\\Documents\\projects\\school\\MetaExtractor\\src\\entity_extraction\\training\\hf_token_classification'

In [ ]:
# ensure src is in path
# sys.path.append(os.path.join(os.getcwd(), os.pardir, os.pardir, os.pardir))

In [6]:
from src.entity_extraction.entity_extraction_evaluation import get_token_labels, load_json_label_files

labelled_file_path = os.path.join(
    os.getcwd(), 
    os.pardir, 
    os.pardir, 
    os.pardir, 
    os.pardir,
    "data",
    "labelled",
    "labelled"
  )

In [14]:
all_text, labelled_entities = load_json_label_files(labelled_file_path)

In [15]:
tokens, token_labels = get_token_labels(labelled_entities, all_text)

In [16]:
id2label = {
    0: "O",
    1: "B-GEOG",
    2: "I-GEOG",
    3: "B-SITE",
    4: "I-SITE",
    5: "B-EMAIL",
    6: "I-EMAIL",
    7: "B-ALTI",
    8: "I-ALTI",
    9: "B-TAXA",
    10: "I-TAXA",
    11: "B-REGION",
    12: "I-REGION",
    13: "B-AGE",
    14: "I-AGE",
}

label2id = {
    "O": 0,
    "B-GEOG": 1,
    "I-GEOG": 2,
    "B-SITE": 3,
    "I-SITE": 4,
    "B-EMAIL": 5,
    "I-EMAIL": 6,
    "B-ALTI": 7,
    "I-ALTI": 8,
    "B-TAXA": 9,
    "I-TAXA": 10,
    "B-REGION": 11,
    "I-REGION": 12,
    "B-AGE": 13,
    "I-AGE": 14,
}

In [17]:
#  convert the labels to ids
token_label_ids = [label2id[label] for label in token_labels]

In [18]:
# split the data into chunks of 128 tokens and labels
chunked_tokens = [tokens[i : i + 128] for i in range(0, len(tokens), 128)]
chunked_token_label_ids = [token_label_ids[i : i + 128] for i in range(0, len(token_label_ids), 128)]
chunked_labels = [token_labels[i : i + 128] for i in range(0, len(token_labels), 128)]

# make each chunk a dict with keys ner_tags and tokens
chunked_data = [
    {
        "ner_tags": chunked_token_label_ids[i], 
        "tokens": chunked_tokens[i],
        "labels": chunked_labels[i]
     } for i in range(len(chunked_tokens))
]
# make the data into a huggingface dataset
dataset = pd.DataFrame(chunked_data)

from datasets import Dataset, DatasetDict
dataset = Dataset.from_pandas(dataset)

In [19]:
# needed as tokenizing adds CLS and SEP tokens, doesn't match labels
# see here for more detail: https://huggingface.co/docs/transformers/tasks/token_classification
# It does:
# 1. Mapping all tokens to their corresponding word with the word_ids method.
# 2. Assigning the label -100 to the special tokens [CLS] and [SEP] so they’re ignored by the PyTorch loss function.
# 3. Only labeling the first token of a given word. Assign -100 to other subtokens from the same word.
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(examples["tokens"], truncation=True, is_split_into_words=True)

    labels = []
    for i, label in enumerate(examples[f"ner_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)  # Map tokens to their respective word.
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:  # Set the special tokens to -100.
            if word_idx is None:
                label_ids.append(-100)
            elif word_idx != previous_word_idx:  # Only label the first token of a given word.
                label_ids.append(label[word_idx])
            else:
                label_ids.append(-100)
            previous_word_idx = word_idx
        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

In [21]:
tokenizer = AutoTokenizer.from_pretrained("allenai/specter2")

In [22]:
tokenized = dataset.map(tokenize_and_align_labels, batched=True)

Map:   0%|          | 0/833 [00:00<?, ? examples/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


In [18]:
len(tokenized["labels"][0])

184

## Split into Train/Test

In [19]:
from sklearn.model_selection import train_test_split    

train, test = train_test_split(tokenized, test_size=0.2, random_state=42)

In [20]:
# create a new huggingface daatset with train/test
train_dataset = Dataset.from_dict(train)
test_dataset = Dataset.from_dict(test)

dataset_dict = DatasetDict({"train":train_dataset,"test":test_dataset})

In [21]:
dataset_dict

DatasetDict({
    train: Dataset({
        features: ['ner_tags', 'tokens', 'labels', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 666
    })
    test: Dataset({
        features: ['ner_tags', 'tokens', 'labels', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 167
    })
})

In [22]:
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)

In [23]:
import evaluate

seqeval = evaluate.load("seqeval")

In [64]:
import numpy as np

label_list = list(label2id.keys())

# labels = [label_list[i] for i in example[f"ner_tags"]]
labels = label_list


def compute_metrics(p):

    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = seqeval.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

In [65]:
from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer

model = AutoModelForTokenClassification.from_pretrained(
    "allenai/specter2", num_labels=len(label_list), id2label=id2label, label2id=label2id
)

Some weights of BertForTokenClassification were not initialized from the model checkpoint at allenai/specter2 and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [67]:
# downgraded to transformers==4.28.0 due to PartialState import error, referenced here and was recommended solution: 
# https://github.com/huggingface/transformers/issues/22816 

training_args = TrainingArguments(
    output_dir="specter2-finetuned-v1",
    no_cuda=False,
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=30,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    push_to_hub=False,
    logging_strategy="epoch",
    logging_steps=1
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset_dict["train"],
    eval_dataset=dataset_dict["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

In [ ]:
results = trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:407: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,0.449200,0.177914,0.368586,0.482192,0.417804,0.951768
2,0.145700,0.136031,0.519075,0.615068,0.563009,0.962013
3,0.092000,0.128652,0.543478,0.684932,0.606061,0.963136
4,0.070900,0.121071,0.588889,0.653425,0.619481,0.967019
5,0.055800,0.128056,0.576708,0.705479,0.634627,0.965569
6,0.043400,0.132955,0.588901,0.712329,0.644761,0.965756
7,0.034300,0.139034,0.604486,0.701370,0.649334,0.966177
8,0.028100,0.137334,0.624088,0.702740,0.661082,0.967955
9,0.021900,0.156823,0.587912,0.732877,0.652439,0.964961
10,0.017700,0.156623,0.601589,0.726027,0.657976,0.967206


/usr/local/lib/python3.10/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-sco

Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,0.449200,0.177914,0.368586,0.482192,0.417804,0.951768
2,0.145700,0.136031,0.519075,0.615068,0.563009,0.962013
3,0.092000,0.128652,0.543478,0.684932,0.606061,0.963136
4,0.070900,0.121071,0.588889,0.653425,0.619481,0.967019
5,0.055800,0.128056,0.576708,0.705479,0.634627,0.965569
6,0.043400,0.132955,0.588901,0.712329,0.644761,0.965756
7,0.034300,0.139034,0.604486,0.701370,0.649334,0.966177
8,0.028100,0.137334,0.624088,0.702740,0.661082,0.967955
9,0.021900,0.156823,0.587912,0.732877,0.652439,0.964961
10,0.017700,0.156623,0.601589,0.726027,0.657976,0.967206


/usr/local/lib/python3.10/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [48]:
type(results)

transformers.trainer_utils.TrainOutput

In [58]:
results[2]

{'train_runtime': 40.713,
 'train_samples_per_second': 16.358,
 'train_steps_per_second': 1.032,
 'total_flos': 89037126453240.0,
 'train_loss': 0.16222034181867326,
 'epoch': 1.0}

# Inference

In [9]:
# os.chdir(os.pardir)
os.getcwd()

'c:\\Users\\tyand\\OneDrive\\Documents\\projects\\school\\MetaExtractor'

In [12]:
# load ner model from scheckpoint
from transformers import AutoModelForTokenClassification, AutoTokenizer, pipeline

checkpoint_path = os.path.join(
    "models", "ner", "specter2-finetuned-v1",
    "checkpoint-252"
)

model = AutoModelForTokenClassification.from_pretrained(checkpoint_path)

In [19]:
from transformers import pipeline

classifier = pipeline(
    "ner",
    model=model,
    tokenizer=tokenizer,
)

In [24]:
text = "Although the single OSL sample at Neyshabour is insufﬁcient to provide a date for the abandonment of the alluvial fan, it does show that abandonment is likely to postdate 24.1 Æ 1.9 ka."

In [25]:
len(text.split(" "))

33

In [26]:
classifier(text)

[{'entity': 'B-REGION',
  'score': 0.7819143,
  'index': 8,
  'word': 'ne',
  'start': 34,
  'end': 36},
 {'entity': 'B-REGION',
  'score': 0.31665814,
  'index': 9,
  'word': '##ys',
  'start': 36,
  'end': 38},
 {'entity': 'I-REGION',
  'score': 0.866823,
  'index': 10,
  'word': '##hab',
  'start': 38,
  'end': 41},
 {'entity': 'I-REGION',
  'score': 0.7716321,
  'index': 11,
  'word': '##our',
  'start': 41,
  'end': 44},
 {'entity': 'B-AGE',
  'score': 0.9783675,
  'index': 40,
  'word': '24',
  'start': 171,
  'end': 173},
 {'entity': 'I-AGE',
  'score': 0.7875793,
  'index': 41,
  'word': '.',
  'start': 173,
  'end': 174},
 {'entity': 'I-AGE',
  'score': 0.94838387,
  'index': 42,
  'word': '1',
  'start': 174,
  'end': 175},
 {'entity': 'I-AGE',
  'score': 0.9709645,
  'index': 43,
  'word': 'æ',
  'start': 176,
  'end': 177},
 {'entity': 'I-AGE',
  'score': 0.9776774,
  'index': 44,
  'word': '1',
  'start': 178,
  'end': 179},
 {'entity': 'I-AGE',
  'score': 0.9708373,
  'in

In [5]:
os.getcwd()

'c:\\Users\\tyand\\OneDrive\\Documents\\projects\\school\\MetaExtractor\\src\\training\\hf_token_classification'

In [18]:
from src.entity_extraction.training.hf_token_classification.labelstudio_preprocessing import convert_labelled_data_to_hf_format 

# use autoreload to reload modules
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


[autoreload of src.entity_extraction.training.hf_token_classification.labelstudio_preprocessing failed: Traceback (most recent call last):
  File "c:\Users\tyand\miniconda3\envs\ffossils\lib\site-packages\IPython\extensions\autoreload.py", line 273, in check
    superreload(m, reload, self.old_objects)
  File "c:\Users\tyand\miniconda3\envs\ffossils\lib\site-packages\IPython\extensions\autoreload.py", line 471, in superreload
    module = reload(module)
  File "c:\Users\tyand\miniconda3\envs\ffossils\lib\importlib\__init__.py", line 169, in reload
    _bootstrap._exec(spec, module)
  File "<frozen importlib._bootstrap>", line 619, in _exec
  File "<frozen importlib._bootstrap_external>", line 883, in exec_module
  File "<frozen importlib._bootstrap>", line 241, in _call_with_frames_removed
  File "c:\Users\tyand\OneDrive\Documents\projects\school\MetaExtractor\src\entity_extraction\training\hf_token_classification\..\..\..\..\src\entity_extraction\training\hf_token_classification\label

In [19]:
labelled_data_path = os.path.join(
    os.pardir, os.pardir, os.pardir, os.pardir,
    "data", "labelled", "labelled"
)

In [20]:
convert_labelled_data_to_hf_format(
    labelled_data_path,
    max_seq_length=512,
    train_split = 0.7,
    val_split=0.15,
    test_split=0.15,
)

In [21]:
gdd_ids = ['55c7e851cf58f1a8110ba2e3', '54b43248e138239d86849e09', '55070990e1382326932d93c8', '5724521fcf58f1bc023df2d2', '5697ec53cf58f1143ae00811', '54b43269e138239d8684f8b5', '54b43283e138239d86854158', '56818c02cf58f1ba274d4652', '5506a7cde1382326932d9244', '5501e1b5e1382326932d7436', '5507ac25e1382326932d9671', '54b4325de138239d8684d700', '54b4326be138239d86850036', '54b4326de138239d8685034c']

In [28]:
    np.random.seed(42)
    # split the gdd_ids into train, val and test and ensure not overlapping

    train_gdd_ids = np.random.choice(
        gdd_ids, size=int(0.7 * len(gdd_ids)), replace=False
    )
    remaining_gdd_ids = np.setdiff1d(gdd_ids, train_gdd_ids)

    val_gdd_ids = np.random.choice(
        remaining_gdd_ids, size=int(0.2 * len(gdd_ids)), replace=False
    )
    remaining_gdd_ids = np.setdiff1d(remaining_gdd_ids, val_gdd_ids)

    test_gdd_ids = np.random.choice(
        remaining_gdd_ids, size=int(0.1 * len(gdd_ids)), replace=False
    )

['55c7e851cf58f1a8110ba2e3', '54b43248e138239d86849e09', '55070990e1382326932d93c8', '5724521fcf58f1bc023df2d2', '5697ec53cf58f1143ae00811', '54b43269e138239d8684f8b5', '54b43283e138239d86854158', '56818c02cf58f1ba274d4652', '5506a7cde1382326932d9244', '5501e1b5e1382326932d7436', '5507ac25e1382326932d9671', '54b4325de138239d8684d700', '54b4326be138239d86850036', '54b4326de138239d8685034c']
['54b43283e138239d86854158' '5507ac25e1382326932d9671'
 '56818c02cf58f1ba274d4652' '5697ec53cf58f1143ae00811'
 '5724521fcf58f1bc023df2d2']
['54b43283e138239d86854158' '5507ac25e1382326932d9671'
 '56818c02cf58f1ba274d4652']


In [30]:
# assert no overlap between train, val and test
assert len(np.intersect1d(train_gdd_ids, val_gdd_ids)) == 0
assert len(np.intersect1d(train_gdd_ids, test_gdd_ids)) == 0
assert len(np.intersect1d(val_gdd_ids, test_gdd_ids)) == 0
